In [1]:
from funcs_ssp_study import *
import os
import xarray as xr
import pandas as pd
import geopandas as gpd
import re
import json
from tqdm import tqdm
import cdsapi
import zipfile
import shutil
import requests

# Code

In [8]:
data_directory = "/Users/michellewan/PycharmProjects/TraCE/data_pipeline"
data_subdirectory = os.path.join(data_directory, "raw")
output_subdirectory = os.path.join(data_directory, "processed_monthly")

country_shapes_filepath = "/Users/michellewan/PycharmProjects/TraCE/data_pipeline/ne_10m_admin_0_countries/ne_10m_admin_0_countries.shp"
country_codes_filepath = "/Users/michellewan/PycharmProjects/TraCE/data_pipeline/OECD_population_GDP/OECD_country_codes.csv"
alignments_filepath = "/Users/michellewan/PycharmProjects/TraCE/alignments.json"

socioeconomic_directory = "/Users/michellewan/PycharmProjects/TraCE/data_pipeline/OECD_population_GDP"
socioeconomic_ssp_filepath = os.path.join(socioeconomic_directory, "SspDb_country_data_2013-06-12.csv")
population_historical_filepath = os.path.join(socioeconomic_directory, "HISTPOP_22082023151824127.csv")
gdp_historical_filepath = os.path.join(socioeconomic_directory, "DP_LIVE_22082023165117703.csv")

In [3]:
climate_variables = [
    "temperature", 
    "pressure",
    "wind_speed",
    "precipitation"
]

socioeconomic_variables = [
    "GDP|PPP", 
    "Population"
]

atmospheric_variables = [
    "ch4"
]

In [4]:
SSPs = [str(ssp) for ssp in range(1, 5+1)]

In [5]:
frequency = "M"

start_year = 2015
end_year = 2023

## DOWNLOAD

In [ ]:
# Downloading climate data from Copernicus CDS requires a registered account (free)
download_climate_variables_data(data_subdirectory, climate_variables, SSPs, start_year, end_year)

In [ ]:
# Atmospheric SSP data is freely available via ESGF
download_atmospheric_variables_data(data_subdirectory, atmospheric_variables, SSPs)

## PROCESS

In [6]:
process_netcdf_data(data_subdirectory, output_subdirectory, 
                    country_shapes_filepath, country_codes_filepath, alignments_filepath, 
                    frequency)

Processing files...
ssp1_ch4.nc


Aggregating: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 258/258 [34:18<00:00,  7.98s/it]


ssp3_wind_speed.nc


Aggregating: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 258/258 [00:08<00:00, 31.82it/s]


ssp4_wind_speed.nc


...

ssp2_wind_speed.nc


Aggregating: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 258/258 [02:30<00:00,  1.72it/s]


ssp4_ch4.nc


Aggregating: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 258/258 [12:56<00:00,  3.01s/it]


Processed data files saved to /Users/michellewan/PycharmProjects/TraCE/data_pipeline/processed_monthly


In [9]:
# Country-level GDP and population SSP data is downloaded from IIASA via a registered account (free)

# Process the socioeconomic datasets
extract_socioeconomic_ssp_data(socioeconomic_ssp_filepath, output_subdirectory, socioeconomic_variables, SSPs, start_year, end_year, frequency)
process_population_historical_data(population_historical_filepath, output_subdirectory, frequency)
process_GDP_historical_data(gdp_historical_filepath, output_subdirectory, frequency)

Extracting socioeconomic data from /Users/michellewan/PycharmProjects/TraCE/data_pipeline/OECD_population_GDP/SspDb_country_data_2013-06-12.csv
Processing GDP|PPP...


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 14.16it/s]


Processing Population...


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 14.17it/s]


Saved output to /Users/michellewan/PycharmProjects/TraCE/data_pipeline/processed_monthly
Processing /Users/michellewan/PycharmProjects/TraCE/data_pipeline/OECD_population_GDP/HISTPOP_22082023151824127.csv
Output saved to /Users/michellewan/PycharmProjects/TraCE/data_pipeline/processed_monthly/historical_population.csv
Processing /Users/michellewan/PycharmProjects/TraCE/data_pipeline/OECD_population_GDP/DP_LIVE_22082023165117703.csv
Saved output to /Users/michellewan/PycharmProjects/TraCE/data_pipeline/processed_monthly/historical_gdp.csv


# Scratch